In [ ]:
!pip install -q nnAudio

In [ ]:
!pip install timm

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
example = np.load('../input/g2net-gravitational-wave-detection/train/0/0/0/00000e74ad.npy')
print(example.shape)
display(pd.DataFrame(example).head())

In [ ]:
train = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
test = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

def get_train_file_path(image_id):
    return "../input/g2net-gravitational-wave-detection/train/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id)

def get_test_file_path(image_id):
    return "../input/g2net-gravitational-wave-detection/test/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id)

train['file_path'] = train['id'].apply(get_train_file_path)
test['file_path'] = test['id'].apply(get_test_file_path)

display(train.head())
display(test.head())

In [ ]:
sns.countplot(x = train['target'])

In [ ]:
fig,a =  plt.subplots(3,1)
a[0].plot(example[1],color='green')
a[1].plot(example[1],color='red')
a[2].plot(example[1],color='yellow')
fig.suptitle('Target 1', fontsize=16)
plt.show()

In [ ]:
i = 5
print(train.columns)
#print(file_paths[:5])

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch
from nnAudio.Spectrogram import CQT1992v2
class G2NetDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df['target'].values
        self.wave_transform = CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=64)
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def apply_qtransform(self, waves, transform):
        waves = np.hstack(waves)
        waves = waves / np.max(waves)
        waves = torch.from_numpy(waves).float()
        image = transform(waves)
        return image

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        waves = np.load(file_path)
        image = self.apply_qtransform(waves, self.wave_transform)
        if self.transform:
            image = image.squeeze().numpy()
            image = self.transform(image=image)['image']
        label = torch.tensor(self.labels[idx]).float()
        return {'image':image,
                'targets': label}

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform
def get_transforms(*, data):
    
    if data == 'train':
        return A.Compose([
            ToTensorV2(),
        ])

    elif data == 'valid':
        return A.Compose([
            ToTensorV2(),
        ])

In [ ]:
from sklearn.model_selection import train_test_split
train_df,valid_df = train_test_split(train,test_size=0.05,random_state=42)

In [ ]:
train_dataset = G2NetDataset(train_df,transform=get_transforms(data='train'))
valid_dataset = G2NetDataset(valid_df,transform=get_transforms(data='train'))

In [ ]:
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset

class CustomModel(nn.Module):
    def __init__(self,pretrained=False):
        super().__init__()
        self.model = timm.create_model('tf_efficientnet_b7_ns', pretrained=pretrained, in_chans=1)
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(self.n_features, 1)

    def forward(self, x):
        output = self.model(x)
        return output

In [ ]:
def train(data_loader, model, optimizer, device):
    # put the model in train mode
    model.train()
    l = len(data_loader)
    step = 0
    # go over every batch of data in data loader
    for data in data_loader:
         # remember, we have image and targets
         # in our dataset class
        inputs = data["image"]
        targets = data["targets"]
         # move inputs/targets to cuda/cpu device
        inputs = inputs.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
         # zero grad the optimizer
        optimizer.zero_grad()
         # do the forward step of model
        outputs = model(inputs)
     # calculate loss
        loss = nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))
     # backward step the loss
        loss.backward()
     # step optimizer
        optimizer.step()
        print(f'{step}/{l},loss={loss}')
        step = step + 1

In [ ]:
def evaluate(data_loader, model, device):
     # put model in evaluation mode
    model.eval()
     # init lists to store targets and outputs
    final_targets = []
    final_outputs = []
     # we use no_grad context
    with torch.no_grad():
        for data in data_loader:
            inputs = data["image"]
            targets = data["targets"]
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            # do the forward step to generate prediction
            output = model(inputs)
            # convert targets and outputs to lists
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()

            # extend the original list
            final_targets.extend(targets)
            final_outputs.extend(output)

     # return final output and final targets
    return final_outputs, final_targets

In [ ]:
device = device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomModel(pretrained = True)
model.to(device)
epochs = 2
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
valid_loader =  torch.utils.data.DataLoader(valid_dataset,batch_size =16, shuffle=True, num_workers=4)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
for epoch in range(epochs):
    train(train_loader, model, optimizer, device=device)
    predictions, valid_targets = evaluate(
     valid_loader, model, device=device)
    roc_auc = metrics.roc_auc_score(valid_targets, predictions)
    print(f"Epoch={epoch}, Valid ROC AUC={roc_auc}")